In [ ]:
import re
import hanja # !pip install hanja
import pandas as pd

train_dataset = pd.read_csv('../../../data/train.csv')
train_texts = list(train_dataset['text'].values)
IDs = list(train_dataset['ID'].values)

korean = re.compile('[^가-힣...…\s]+') # 한국어, ..., …, 공백 외의 문자가 한 번 이상 등장할 경우에 대한 패턴

train_dataset['polluted_lv'] = None
train_dataset['no_hanja'] = None
for id_ in IDs:
    text = train_dataset[train_dataset['ID']==id_]['text'].values[0]
    text = hanja.translate(text, 'substitution')
    text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", text)
    
    results = korean.findall(text)
    total = sum([len(r) for r in results])
    prob = total / len(text)
    train_dataset.loc[train_dataset['ID']==id_, 'polluted_lv'] = prob
    train_dataset.loc[train_dataset['ID']==id_, 'no_hanja'] = text

# train_dataset.to_csv('train-no_special-no_hanja-polluted_lv.csv')

ModuleNotFoundError: No module named 'hanja'

In [87]:
# train = pd.read_csv('train-no_special-no_hanja-polluted_lv.csv').drop(columns=['Unnamed: 0'])
train_dataset[(train_dataset['target']==6) & (0.30<train_dataset['polluted_lv']) & (train_dataset['polluted_lv']<0.40)].sample(5)

NameError: name 'train_dataset' is not defined

### LLaMA로 거지같은 텍스트 골라내기

In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B' # 'beomi/Llama-3-Open-Ko-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto'
)

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): L

### 텍스트 클렌징

In [ ]:
PROMPT = '''주어진 데이터의 문맥이 어색한 부분을 교정하세요.
1. 문맥이 어색하다면 교정하세요.
2. 어색하지 않다면 그대로 출력하세요.
3. 복원할 수 없다면 '복원 불가'라고 하세요.'''

fewshot = [
    {"role": "user", "content": "올림픽|N데월드몰 0니j컬링y경R s벤0 진행"},
    {"role": "assistant", "content": "올림픽 롯데월드몰 미니 컬링 경품 이벤트 진행"},
    {"role": "user", "content": "CJ헬로비전SKB 합병 무효 소송 잇따라…LGU＋ 법적 대응"},
    {"role": "assistant", "content": "CJ헬로비전SKB 합병 무효 소송 잇따라…LGU＋ 법적 대응"},
    {"role": "user", "content": "정i 파1 미사z KT 이용기간 2e 단 Q분종U2보"},
    {"role": "assistant", "content": "복원 불가"}
]

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

for data in train.iterrows():
    data = data[1]
    messages = [{"role": "system", "content": PROMPT}] + fewshot + [{"role": "user", "content": data['text']}]

    input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors='pt'
        ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    print(data['text'])
    print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))
    print()

### 키워드 추출

In [ ]:
PROMPT = '''당신은 기사 제목을 보고 어떤 분야의 기사인지 맞추는 전문가입니다.
[지시사항]
1. 주어진 데이터는 개행 기호(\\n)로 구분된 뉴스 기사 제목들입니다.
2. 주어진 데이터에는 임의의 자리에 문맥에 맞지 않는 글자가 무작위로 삽입되어 있습니다.
3. 해당 데이터들을 가장 잘 표현하는 포괄적인 기사 분야를 한 단어로만 출력하세요.'''

fewshot = [
    {"role": "user", "content": "SW최Z 19도 날v 따뜻O져중부 밤늦게 m 조금"},
    {"role": "assistant", "content": "생활"},
    {"role": "user", "content": "게Z판 통 2 편입생 모집 0카c톡bb진행"},
    {"role": "assistant", "content": "사회"},
    {"role": "user", "content": "말로하E 대화형 내비 나왔다S8T Tnx구출시종"},
    {"role": "assistant", "content": "기술"},
    {"role": "user", "content": "T치K2 Y루마니아 방문정 회R 함y 기도"},
    {"role": "assistant", "content": "해외"}
]

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

keywords = []
for i in range(7):
    keys = []
    for j in range(10):
        data = train[(train['target']==i) & (0.30<train['polluted_lv']) & (train['polluted_lv']<0.40)].sample(5)['no_hanja'].values
        data = '\n'.join(data)

        messages = [{"role": "system", "content": f"{PROMPT}"}] + fewshot + [{"role": "user", "content": f"{data}"}]

        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors='pt'
        ).to(model.device)

        outputs = model.generate(
            input_ids,
            max_new_tokens=1024,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9
        )
        keys.append(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

    keywords.append(keys)

keywords

[['스포츠', '사회', '문학', '영화', '문화', '언론', '경제', '정치', '기상', '문학'],
 ['스포츠', '스포츠', '스포츠', '스포츠', '스포츠', '스포츠', '스포츠', '스포츠', '스포츠', '스포츠'],
 ['정치', '정치', '정치', '정치', '정치', '정치', '정치', '정치', '정치', '외교'],
 ['사회', '교육', '법', '교육', '사회', '사회', '사회', '기술', '교육', '경제'],
 ['기술', '기술', '기술', '기술', '기술', '기술', '전자', '전자', '전자', '기술'],
 ['경제', '경제', '경제', '산업', '금융', '경제', '경제', '경제', '경제', '경제'],
 ['외교', '경제', '정치', '정치', '경제', '정치', '경제', '외교', '경제', '미국']]

In [86]:
PROMPT = '''제공된 데이터들은 뉴스 기사 분야들입니다. 전체 단어를 아우를 수 있는 하나의 키워드를 한 단어로 출력하세요.'''
fewshot = [
    {"role": "user", "content": "외교, 정치, 경제, 외교, 경제, 정치, 외교, 외교, 외교, '정치"},
    {"role": "assistant", "content": "외교"}
]

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

final_keys = []
for keys in keywords:
    keys = ', '.join(keys)

    messages = [{"role": "system", "content": f"{PROMPT}"}] + fewshot + [{"role": "user", "content": f"{keys}"}]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors='pt'
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )
    final_keys.append(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

final_keys

['문학', '스포츠', '정치', '사회', '기술', '경제', '외교']